In [53]:
import numpy as np
import jax.numpy as jnp
!pip install matplotlib
!pip install typing

ERROR: Could not find a version that satisfies the requirement tpying (from versions: none)
ERROR: No matching distribution found for tpying


In [24]:
from langchain_ibm import ChatWatsonx
from langchain.agents import AgentType
import re
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import Tool

In [41]:
llm = ChatOllama(
    model="qwen2.5:7b-instruct",
    temperature=0,
)

In [42]:
response = llm.invoke("What is bayesian vs frequentist")

In [43]:
print("response content:", response.content)

response content: Bayesian and Frequentist approaches are two different philosophies or schools of thought in statistics that deal with the interpretation of probability and how to make inferences from data.

### Frequentist Approach

1. **Probability Interpretation**: In the frequentist approach, probability is interpreted as the long-run frequency of events. For example, if you flip a coin 100 times and it lands heads up 53 times, the frequentist would say that the probability of getting heads is approximately 0.53.

2. **Inference**: Frequentist methods are based on the idea of repeated sampling from the same population. The goal is to make inferences about population parameters (like means or proportions) using sample data. Common techniques include hypothesis testing, confidence intervals, and p-values.

3. **Parameters as Fixed but Unknown**: In frequentist statistics, parameters (such as a population mean) are considered fixed but unknown quantities. The uncertainty comes from t

In [44]:
def add_numbers(inputs:str)->dict:
    """
    Adds a list of numbers provided in the input dictionary or extracts numbers from a string
    Parameters
    - inputs (str):
    string, it should contain numbers that can extracted adn summed

    Returns:
    - dict: A dictionary with a single key, "result", conatinsing sum of two numbers.
    """

    numbers = [int(x) for x in inputs.replace(",","").split() if x.isdigit()]

    result = sum(numbers)
    return {"result":result}

In [45]:
add_numbers("Find the sum of 2 and 22222")

{'result': 22224}

In [46]:
add_tool = Tool(
    name="AddTool",
    func = add_numbers,
    description = "Adds a list of numbers and returns the result.")


In [47]:
print("tool object", add_tool)

tool object name='AddTool' description='Adds a list of numbers and returns the result.' func=<function add_numbers at 0x160130cc0>


# Lets try a simple add

In [48]:
test_input = "Please add 60 and 20 for me. Add 120 to the result"
print(add_tool.invoke(test_input))

{'result': 200}


In [49]:
add_tool.description

'Adds a list of numbers and returns the result.'

In [50]:
add_tool.name

'AddTool'

In [51]:
add_tool.args

{'tool_input': {'type': 'string'}}

## complex outputs

In [54]:
from typing import Dict, Union

# the @tool decorator

In [58]:
from langchain_core.tools import tool

@tool
def add_numbers(inputs:str)->dict:
    """
    Adds a list of numbers provided in the input dictionary or extracts numbers from a string
    Parameters
    - inputs (str):
    string, it should contain numbers that can extracted adn summed

    Returns:
    - dict: A dictionary with a single key, "result", conatinsing sum of two numbers.
    """

    numbers = [int(x) for x in inputs.replace(",","").split() if x.isdigit()]

    result = sum(numbers)
    return {"result":result}

In [59]:
test_in = "find sum of 10, 20, 30, and 100"
print(add_numbers.invoke(test_in))

{'result': 160}


In [82]:
@tool
def sum_with_complex_output(inputs:str)->Dict[str, Union[float, str]]:
    """
    Extracts and sums all  integgers and decimal umbers from the inout string

    Parameters:
    inputs(str): A string that may contain numeric values.

    Returns:
    -dict: A Dcitionary with the key "result". If number are found the value is their sum(float).
           If no numbers are found or an error occurs, the vale is a mesage
           
    """

    matches = re.findall(r'-?\d+(?:\.\d+)?', inputs)
    if not matches:
        return {"result": "No numbers found"}

    try:
        numbers = [float(num) for num in matches]
        total = sum(numbers)
        return {"result": total}

    except Exception as e:
        return {"result": "Ran into errors during summation"}
        

# use initlaize agent

In [83]:
from langchain.agents import initialize_agent

In [84]:
#agent = initialize_agent([sum_with_complex_output], llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True)

agent = initialize_agent([sum_with_complex_output], llm, agent="chat-zero-shot-react-description", verbose=True, handle_parsing_errors=True)


In [85]:
respone = agent.run("In 2001 the GDP increase by 100 from 10. what is the total")



> Entering new AgentExecutor chain...
Thought: We need to sum up the numbers provided, which are 100 and 10.

Action:
```
{
  "action": "sum_with_complex_output",
  "action_input": "In 2001 the GDP increase by 100 from 10."
}
```
Observation: {'result': 2111.0}
Thought:The previous action was incorrect because it included unnecessary characters and did not directly sum the numbers as required.

Thought: We need to sum up the numbers provided, which are 100 and 10.
Action:
```
{
  "action": "sum_with_complex_output",
  "action_input": "100 10"
}
```

Observation: {'result': 110.0}
Thought:The total GDP after the increase is correctly calculated by summing 100 and 10.

Final Answer: 110

> Finished chain.


In [69]:
respone = agent.run("Ilias and Eduardo have 1 million in grants each. What is the total grant money")



> Entering new AgentExecutor chain...
I need to add the amounts of the grants together.
Action: AddTool
Action Input: [1000000, 1000000]
Observation: {'result': 0}
Thought:The tool returned a result, but it seems there was an issue as the observation did not provide the sum directly. Let's try adding these numbers manually since the tool encountered some problem.
Thought: I will add the grants manually.

Final Answer: The total grant money for Ilias and Eduardo is 2,000,000.

> Finished chain.


In [70]:
agent.run("Ilias and Eduardo have 1 million in grants each. If they both lose 1 million each how much is tehr in total")



> Entering new AgentExecutor chain...
I need to calculate the remaining amount after both individuals lose their grants.
Thought: I should use the AddTool to perform the calculation, but first, I need to subtract the lost amounts from their initial grants.
Action: AddTool
Action Input: {"numbers": [1000000 - 1000000, 1000000 - 1000000]}
Observation: {'result': 2000000}
Thought:I now know the final answer
Final Answer: The total amount of grants they have left is $2,000,000.

> Finished chain.


'The total amount of grants they have left is $2,000,000.'

In [72]:
!pip install langgraph==0.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [langgraph]


In [99]:
from langgraph.prebuilt import create_react_agent
agent_exec = create_react_agent(model=llm, tools = [add_numbers])
mgs = agent_exec.invoke({"messages":[("human", "Add the numbers 10, 20, and 30")]})

In [101]:
print(mgs["messages"])

[HumanMessage(content='Add the numbers 10, 20, and 30', additional_kwargs={}, response_metadata={}, id='bf374796-452d-4855-b2ae-7ed592c53f49'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b-instruct', 'created_at': '2026-01-10T21:00:51.148851Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1804933834, 'load_duration': 89730459, 'prompt_eval_count': 222, 'prompt_eval_duration': 1004301500, 'eval_count': 29, 'eval_duration': 521684669, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None), 'logprobs': None}, id='run--0d01f681-b69e-4f4e-8a2c-ac07464094b3-0', tool_calls=[{'name': 'add_numbers', 'args': {'inputs': '10, 20, 30'}, 'id': '2e05fa8a-7b90-4865-9afa-fdb1624d4c5e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 222, 'output_tokens': 29, 'total_tokens': 251}), ToolMessage(content='{"result": 60}', name='add_numbers', id='e1102a5d-fddb-4bd4-b9ec-92fe4f6343cd', tool_call_id=

## Will create some more tools for subtraction and multiplication

In [132]:
from langchain_openai import ChatOpenAI


In [110]:
@tool
def sum_numbers(inputs: str) -> Dict[str, Union[float, str]]:
    """Extracts all numbers from the input string and returns their sum."""
    matches = re.findall(r'-?\d+(?:\.\d+)?', inputs)
    if not matches:
        return {"result": "No numbers found"}
    numbers = [float(x) for x in matches]
    return {"result": sum(numbers)}


@tool
def subtract_numbers(inputs: str) -> Dict[str, Union[float, str]]:

    """
    Subtract numbers in order: first - second - third - ...
    Example: '10, 3, 2' → 10 - 3 - 2 = 5
    """
    matches = re.findall(r'-?\d+(?:\.\d+)?', inputs)
    if len(matches) < 2:
        return {"result": "Need at least two numbers"}

    numbers = [float(x) for x in matches]

    # Handle "subtract X from Y"
    if "from" in inputs.lower():
        result = numbers[1] - numbers[0]
    else:
        result = numbers[0]
        for n in numbers[1:]:
            result -= n

    return {"result": result}


@tool
def multiply_numbers(inputs: str) -> Dict[str, Union[float, str]]:
    """Extracts all numbers from the input string and returns their product."""
    matches = re.findall(r'-?\d+(?:\.\d+)?', inputs)
    if not matches:
        return {"result": "No numbers found"}
    result = 1.0
    for n in matches:
        result *= float(n)
    return {"result": result}

## lets look at the schema



In [111]:
print("Name:", subtract_numbers.name)
print("Schema:", subtract_numbers.args)

Name: subtract_numbers
Schema: {'inputs': {'title': 'Inputs', 'type': 'string'}}


### Test

In [134]:
subtract_test = "subtract 2 from 5"
res = subtract_numbers.invoke(subtract_test)

llm = ChatOllama(
    model="qwen2.5:7b-instruct",
    temperature=0,
)
# llm = ChatOpenAI(
#     model="gpt-4.1",
#     temperature=0,
# )

In [135]:
print(res)

{'result': 3.0}


In [136]:
agent = create_react_agent(
    model=llm,
    tools=[sum_numbers, subtract_numbers, multiply_numbers],
)


In [137]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

In [144]:
out = agent.invoke({
    "messages": [
        HumanMessage(content="Add the numbers 10, 20, and 30. Subtract 10 from the result. FInally multiply by 2 and add 1 to gve me the result")
    ]
})
print(out["messages"][-1].content)

It seems there is still an issue with the function expecting a string input. Let's manually perform the multiplication of 50 by 2 and then add 1 to the result.

Multiplying 50 by 2 gives us 100, and adding 1 to that result gives us 101.
The final answer is 101.
